In [13]:
import numpy as np
from prettytable import PrettyTable
#set seed
np.random.seed(4)

# Data sampling functions

In [14]:
#Sample the linear predictor, by first sampling the mean from a Redamacher distribution, and then sampling the predictor from a normal distribution around the mean
# dim: the dimension of the data (+ 1 for the constant 1)
# sigma: the standard deviation of the normal distribution
# bias_factor: the scale of the Redamacher distribution
def sample_predictor(dim, sigma, bias_factor):
    mean = (np.random.binomial(1, 0.5, dim+1)*2 - 1)*bias_factor
    return np.random.normal(mean, sigma)

#Sample the data, by sampling from a normal distribution with mean zero and the given standard deviation, and add the constant 1
# dim: the dimension of the data
# data_scale: the standard deviation of the normal distribution
# num_samples: the number of samples
def sample_data(dim, data_scale, num_samples):
    mean = np.zeros(dim)
    data = np.random.normal(mean, data_scale, (num_samples, dim))
    data = np.append(data, np.ones((data.shape[0], 1)), axis=1)
    return data

#Sample the labels, by first computing the probability of the positive class according to the logistic distribution, and then sampling from a Bernoulli distribution with the corresponding probability
# data: the data
# predictor: the linear predictor of the logistic distribution
def sample_labels(data, predictor):
    prob = 1/(1+np.exp(-np.dot(data, predictor)))
    labels = np.random.binomial(1, prob)
    return labels

#Sample the data, a predictor, and the labels using the above functions
def sample_data_and_labels(data_parameters):
    predictor = sample_predictor(data_parameters['dim'], data_parameters['init_sigma'], data_parameters['bias_factor'])
    
    training_data = sample_data(data_parameters['dim'], data_parameters['data_scale'], data_parameters['training_sample_size'])
    training_labels = sample_labels(training_data, predictor)
    training_set = {'data': training_data, 'labels': training_labels}

    test_sample_size = int(data_parameters['training_sample_size'] * data_parameters['test_ratio'])
    test_data = sample_data(data_parameters['dim'], data_parameters['data_scale'], test_sample_size)
    test_labels = sample_labels(test_data, predictor)
    test_set = {'data': test_data, 'labels': test_labels}
    
    full_data = {'training': training_set, 'test': test_set}
    return full_data, predictor

# Logistic regression functions

In [15]:
#Compute the logistic loss of the given predictor on the given data set
# data_set: a dictionary containing the data and labels
# predictor: the linear predictor of the logistic distribution

def logistic_loss(data_set, predictor):
    #clip the probability to avoid numerical instability
    prob = np.clip(1/(1+np.exp(-np.dot(data_set['data'], predictor))), 1e-4, 1-1e-4)
    return -np.mean(data_set['labels'] * np.log(prob) + (1-data_set['labels']) * np.log(1-prob))

#Compute the logistic loss of the given predictor on the given data set using gradient descent
# starting_predictor: the initial predictor
# data_set: a dictionary containing the data and labels
# learning_parameters: a dictionary containing the learning parameters
def logistic_regression(starting_predictor, data_set, learning_parameters):
    predictor = starting_predictor.copy()
    for i in range(learning_parameters['num_of_iterations']):
        prob = np.clip(1/(1+np.exp(-np.dot(data_set['data'], predictor))), 1e-4, 1-1e-4)
        full_gradient = data_set['data'] * (data_set['labels'] - prob)[:, np.newaxis]
        gradient = np.mean(full_gradient, axis=0)
        predictor += learning_parameters['learning_rate'] * gradient
    return predictor

# Simulation infrastructure

In [16]:
# A class to store the losses of different predictors
class Losses:

    def __init__(self, name):
        self.name = name
        self.loss = {}
    
    def __init__(self, name, data_set = [], predictors = []):
        self.name = name
        self.loss = {}
        for key in predictors:
            self.loss[key + ' predictor'] = logistic_loss(data_set, predictors[key])

    def add_loss(self, other):
        for key in other.loss:
            if key in self.loss:
                self.loss[key] += other.loss[key]
            else:
                self.loss[key] = other.loss[key]

    def average_loss(self, num_of_experiments):
        for key in self.loss:
            self.loss[key] /= num_of_experiments

In [17]:
# A single experiment, which samples the data and the predictor, and then computes the losses of the true predictor, the guess predictor, and the learned predictors, one per learning_parameters
# data_parameters: a dictionary containing the data sampling parameters
# learning_parameters_array: an array of dictionaries, each containing a set of learning parameters
def run_experiment(data_parameters, learning_parameters_array):
    full_data, true_predictor = sample_data_and_labels(data_parameters)
    guess_predictor = sample_predictor(data_parameters['dim'], data_parameters['init_sigma'], data_parameters['bias_factor'])
    predictors = {'true': true_predictor, 'guess': guess_predictor}
    for i in range(len(learning_parameters_array)):
        predictors[(learning_parameters_array[i])['name']] = logistic_regression(guess_predictor, full_data['training'], learning_parameters_array[i])

    training_losses = Losses('training', full_data['training'], predictors)
    test_losses = Losses('test', full_data['test'], predictors)
    return training_losses, test_losses

# A full experiment, which runs num_of_experiments experiments using the previous function and averages the losses
# data_parameters: a dictionary containing the data sampling parameters
# learning_parameters_array: an array of dictionaries, each containing a set of learning parameters
# num_of_experiments: the number of experiments
def run_full_experiment(data_parameters, learning_parameters_array, num_of_experiments):
    training_losses = Losses('training')
    test_losses = Losses('test')

    for i in range(num_of_experiments):
        iter_training_losses, iter_test_losses = run_experiment(data_parameters, learning_parameters_array)
        training_losses.add_loss(iter_training_losses)
        test_losses.add_loss(iter_test_losses)

    training_losses.average_loss(num_of_experiments)
    test_losses.average_loss(num_of_experiments)            
    return training_losses, test_losses

def float2txt(num):
    return "{:.3f}".format(num)

#Print the results of the experiment
# data_parameters: a dictionary containing the data sampling parameters
# training_losses: the losses on the training set
# test_losses: the losses on the test set
def print_results(data_parameters, training_losses, test_losses):
    print('Losses for dim = ', str(data_parameters['dim']), ', data_scale = ', str(data_parameters['data_scale']), ', training_sample_size = ', str(data_parameters['training_sample_size']))
    
    table = PrettyTable(['predictor', 'Training', 'test'])
    for key in training_losses.loss:
        table.add_row([key, float2txt(training_losses.loss[key]), float2txt(test_losses.loss[key])])
    print(table)

In [18]:
data_parameters = {'dim': 10, 'data_scale': 2, 'init_sigma': 0.5, 'bias_factor': 1, 'training_sample_size': 300, 'test_ratio': 0.5}
learning_parameters = [{'name': 'Learned', 'learning_rate': 1, 'num_of_iterations': 40}]
num_of_experiments   = 1000

training_losses, test_losses = run_full_experiment(data_parameters, learning_parameters, num_of_experiments)
print_results(data_parameters, training_losses, test_losses)

Losses for dim =  10 , data_scale =  2 , training_sample_size =  300
+-------------------+----------+-------+
|     predictor     | Training |  test |
+-------------------+----------+-------+
|   true predictor  |  0.180   | 0.179 |
|  guess predictor  |  2.569   | 2.568 |
| Learned predictor |  0.170   | 0.199 |
+-------------------+----------+-------+
